<a href="https://colab.research.google.com/github/Rob174/Astronomy/blob/Astronomy/Autoencodeur_N_mean_1_10e_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd '/content/drive/My Drive/TIPE'
import os

/content/drive/My Drive/TIPE


In [0]:
print("Vérification GPU")
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print()
print()
from google.colab import files
import tensorflow as tf
import numpy as np
import scipy
from PIL import Image

# Ajoute images
entrainement = 60/100
test = 20/100
# validation = le reste : 20%
tailleImage = 399

images = []
noises = []

i = 1
while os.path.isfile("Images_source/clean/"+str(i)+".jpg"):
    images.append("Images_source/clean/"+str(i)+".jpg")
    i += 1

i = 1
while os.path.isfile("Images_source/noise/"+str(i)+".jpg"):
    noises.append("Images_source/noise/"+str(i)+".jpg")
    i += 1

Vérification GPU
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14685801412873095619
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 3882054694422592756
physical_device_desc: "device: XLA_CPU device"
]




In [0]:
import cv2
import numpy as np
def next_batch(batch_size, noises, images,tailleAttendue):#ATTENTION : pr tenter d'améliorer l'apprentissage, on augmente la taille minimale d'image prise
    '''
    Return a total of `num` random samples and labels. 
    '''
#     print("Size of {}".format(len(noises)))
    noiseTensor = []
    imageTensor = []
#     import matplotlib.pyplot as plt
    for i in range(batch_size):
      choix = np.random.randint(0,len(images))#choix aléatoire de l'image
      image = cv2.imread(images[choix],1)#Ouvre en rgb l'image nettoyée
      noise = cv2.imread(noises[choix],1)#Ouvre en rgb l'image bruitée
      angle = np.random.randint(0,90)
      taille = np.random.randint(int(tailleAttendue*2**0.5)*4, image.shape[0]+1)# Racine de deux pour pouvoir toujours récupérer la taille désirée
      ##ATENTION : ci-dessus peut porter à erreur si dépasse image.shape[0]+1
      resizedImage = cv2.resize(image,(taille,taille))
      resizedNoise = cv2.resize(noise,(taille,taille))
      
      rows,cols = resizedImage.shape[:2]
#       print('CentreRow : ')
#       print((taille//2,rows-taille//2))
#       print('CentreCols : ')
#       print((taille//2,cols-taille//2))
#       tmp = taille//2
      
      centre = (np.random.randint(taille//2,rows-taille//2) if taille//2<rows-taille//2 else taille//2,np.random.randint(taille//2,cols-taille//2) if taille//2<cols-taille//2 else taille//2)
      M = cv2.getRotationMatrix2D(centre,angle,1)

      rotatedImage = cv2.warpAffine(resizedImage,M,(cols,rows))
      rotatedNoise = cv2.warpAffine(resizedNoise,M,(cols,rows))
#       print('Rotation : '+str(rotatedImage.shape))

      resultImage = rotatedImage[centre[0]-tailleAttendue//2:centre[0]+tailleAttendue//2+1,centre[1]-tailleAttendue//2:centre[1]+tailleAttendue//2+1]
      resultNoise = rotatedNoise[centre[0]-tailleAttendue//2:centre[0]+tailleAttendue//2+1,centre[1]-tailleAttendue//2:centre[1]+tailleAttendue//2+1]
#       print('Crop : '+str(resultImage.shape))
#       plt.imshow(resultImage)

      noiseTensor.append(np.array(resultNoise,np.uint8))
      imageTensor.append(np.array(resultImage,np.uint8))
#       print(imageTensor)
#     plt.show()
    noiseTensor,imageTensor = np.array(noiseTensor,np.float32), np.array(imageTensor,np.float32)
    return noiseTensor, imageTensor
# next_batch(3,noises,images,200)

In [0]:
for i in range(10):
  var = next_batch(3,noises,images,200)
  print(np.min(var),np.max(var))

0.0 255.0
0.0 255.0
0.0 255.0
0.0 253.0
0.0 255.0
0.0 255.0
0.0 255.0
0.0 255.0
0.0 255.0
0.0 255.0


In [0]:
height = 199
width = 199
channels = 3
forceInit = True

batch_size = 7
learning_rate_power = 2
learning_rate = 10**-learning_rate_power
i = 1
nom = 'Autodencodeur_N_'+'mean'+'_'+str(i)+'_10e-'+str(learning_rate_power)

tf.reset_default_graph()
import traceback

root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)
with tf.Graph().as_default():
  
    def denseLayer(input,nbNeurones,nom,start=False,end=False):
      couche = tf.layers.dense(input,nbNeurones,name=nom)
      print('Taille de '+nom+' : '+str(couche.get_shape()))
      kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
      bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
      kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
      bias_saver = tf.summary.histogram("Bias/"+nom,bias)
      return couche,kernel,bias,kernel_saver,bias_saver
    
    def lrelu(x, leak=0.2): 
      return tf.maximum(x, leak * x)
    
    def convLayer(inpt,kernelListOrNot,num_filters,poolingSize,poolingType,nom,start=False,end=False,generator=False):
      def conv(inpt,kernel,num_filters,poolingSize,poolingType,nom,start=False,end=False,pooling=True):
        couche = tf.layers.conv2d(inpt, filters=num_filters, kernel_size=kernel,
             strides=1, padding='SAME',
             activation=tf.nn.relu, name=nom)
        print('Taille de '+nom+' : '+str(couche.get_shape()))
        kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
        bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
        kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
        bias_saver = tf.summary.histogram("Bias/"+nom,bias)
        couche = tf.contrib.layers.batch_norm(couche, is_training=training, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope=nom+'_normalization')
        if pooling == True:
          if poolingType == 'mean':
            couche = tf.layers.average_pooling2d(lrelu(couche), pool_size=(poolingSize,poolingSize), strides=(2,2), padding='same')
          else:
            couche = tf.layers.max_pooling2d(lrelu(couche), pool_size=(poolingSize,poolingSize), strides=(2,2), padding='same')
        return couche,kernel,bias,kernel_saver,bias_saver
      if type(kernelListOrNot) == list:
        couche,kernel,bias,kernel_saver,bias_saver = conv(inpt,kernelListOrNot[0],num_filters,poolingSize,poolingType,nom+'_split_0',start,end,pooling=False)
        kernelList,biasList,kernel_saverList,bias_saverList = [kernel],[bias],[kernel_saver],[bias_saver]
        for i in range(1,len(kernelListOrNot)):
          couche,kernel,bias,kernel_saver,bias_saver = conv(couche,kernelListOrNot[i],num_filters,poolingSize,poolingType,nom+'_split_'+str(i),False,False if i != len(kernelListOrNot)-1 else end,pooling=False if i != len(kernelListOrNot)-1 else True)
          kernelList.append(kernel)
          biasList.append(bias)
          kernel_saverList.append(kernel_saver)
          bias_saverList.append(bias_saver)
        return couche,kernelList,biasList,kernel_saverList,bias_saverList
      else: 
        return conv(inpt,kernelListOrNot,num_filters,poolingSize,poolingType,nom,start,end)
      
    def deconvLayer(inpt,sizeListOrNot,num_filters,nom,start=False,end=False):
      """Ceci est une documentation"""
      def deconv(inpt,size,num_filters,nom,start=False,end=False):
        def calculateParameters(precDimension,outputSize, strides): # Ref : https://adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/
#           print(precDimension)
          return outputSize-(precDimension-1)*strides
        kernelSize = calculateParameters(inpt.get_shape()[1],size,1)
        print('Taille du noyau de deconvolution de '+nom+' : ' +str(kernelSize))
        couche = tf.layers.conv2d_transpose(inpt, filters=num_filters, kernel_size=(kernelSize,kernelSize),
             strides=1, padding='VALID',
             activation=tf.nn.relu, name=nom)
        print('Taille de '+nom+' : '+str(couche.get_shape()))
        kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
        bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
        kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
        bias_saver = tf.summary.histogram("Bias/"+nom,bias)
        couche = tf.contrib.layers.batch_norm(couche, is_training=training, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope=nom+'_normalization')
        
        return couche,kernel,bias,kernel_saver,bias_saver
      if type(sizeListOrNot) == list:
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconv(inpt,sizeListOrNot[0],num_filters,poolingSize,poolingType,nom+'_split_0',True,False)
        kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList = [kernel],[bias],[kernel_saver],[bias_saver],[sortieImage]
        for i in range(1,len(sizeListOrNot)):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconv(couche,sizeListOrNot[i],num_filters,nom+'_split_'+str(i),False,False if i != len(sizeListOrNot)-1 else True)
          kernelList.append(kernel)
          biasList.append(bias)
          kernel_saverList.append(kernel_saver)
          bias_saverList.append(bias_saver)
        return couche,kernelList,biasList,kernel_saverList,bias_saverList
      else: 
        return deconv(inpt,sizeListOrNot,num_filters,nom,start,end)
    def generateur(inpt,kernels,num_filters,pooling,poolingTypes):
      """
      Description générale : Fonction créant le générateur, encodeur d'informations de l'image
      Entree : 
        inpt, couche d'entrée, ici ce sera l'image bruitée
        kernels, les différents noyaux de convolution, se présente soit sous forme de 
                liste simple soit sous forme d'une double liste. 
                Chaque sous liste représente un noyau de convolution 
                trop gros pour être réalisé en une couche et qui est donc séparé en sous-couches
        num_filters, la taille de filtre pour la couche (en gardant le même nombre de filtre
                pour chaque sous-couche si on a séparé la couche en sous-couches)
        poolingSize, les tailles des noyaux des couche de pooling
        poolingType, le type de pooling utilisé : SAME en générale
      Sortie : 
        L'image encodée (même si ce n'est plus réellement une image maintenant)
      """
      kernelBiasList,saver= [],[]
      def recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,kernelBiasList,saver):
        if type(kernel) == list:
          for k in kernel:
            kernelBiasList.append(k)
          for b in bias:
            kernelBiasList.append(b)
          for ks in kernel_saver:
            saver.append(ks)
          for bs in bias_saver:
            saver.append(bs)
        else:
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
        return None #Utilise la mutabilité des listes
      couche,kernel,bias,kernel_saver,bias_saver = convLayer(inpt,kernels[0],num_filters[0],pooling[0],poolingTypes[0],'convGenerator0',start=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,kernelBiasList,saver)
      for i in range(1,len(kernels)-1):
        couche,kernel,bias,kernel_saver,bias_saver = convLayer(couche,kernels[i],num_filters[i],pooling[i],poolingTypes[i],'convGenerator'+str(i))
        recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,kernelBiasList,saver)

      couche,kernel,bias,kernel_saver,bias_saver = convLayer(couche,kernels[-1],num_filters[-1],pooling[-1],poolingTypes[-1],'convGenerator'+str(len(kernels)-1),end=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,kernelBiasList,saver)
      return couche,kernelBiasList,saver

    def discriminator(inpt,size,num_filters,ID):
      """
      Entrees : 
        inpt, couche d'entrée du discriminateur
        size, liste, ou liste de liste si décomposition les tailles de couche successives pour revenir à la taille initiale
        num_filters, le nombre de filtre par couches
        ID, id unique pour séparer deux potentiels discriminateurs
      Sortie :
        Couche traitée par le discriminateur
      """
      kernelBiasList,saver= [],[]
      print("Taille de l'entrée du discriminateur : ",end='')
      print(inpt.get_shape())
      def recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,kernelBiasList,saver):
        if type(kernel) == list:
          for k in kernel:
            kernelBiasList.append(k)
          for b in bias:
            kernelBiasList.append(b)
          for ks in kernel_saver:
            saver.append(ks)
          for bs in bias_saver:
            saver.append(bs)
        else:
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
        return None #Utilise la mutabilité des listes
      couche,kernel,bias,kernel_saver,bias_saver = deconvLayer(inpt,size[0],num_filters[0],'deconvDiscriminator'+ID+'0',start=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,kernelBiasList,saver)
      for i in range(1,len(size)-1):
        couche,kernel,bias,kernel_saver,bias_saver = deconvLayer(couche,size[i],num_filters[i],'deconvDiscriminator'+ID+str(i))
        recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,kernelBiasList,saver)

      couche,kernel,bias,kernel_saver,bias_saver = deconvLayer(couche,size[-1],num_filters[-1],'deconvDiscriminator'+ID+str(len(size)-1),end=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,kernelBiasList,saver)
      return couche,kernelBiasList,saver
    def discriminatorDense(inpt,neurones,ID):#ID car besoin nom unique pour chaque série de discriminator
        saver = []
        kernelBiasList = []
        sortiesImages = []
        couche = inpt
        for i,nb in enumerate(neurones):
          couche,kernel,bias,kernel_saver,bias_saver = denseLayer(couche,nb,'dense'+str(ID)+'_'+str(i+1))
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
        return couche,kernelBiasList,saver
      
    gen_input = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="input_noise")
    disc_input = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="input_clean")
    training = tf.placeholder_with_default(False, shape=(), name='training')
    print('Taille de gen_input : '+str(gen_input.get_shape()))
    
    import numpy as np
    
    generator,gen_vars,gen_saver = generateur(gen_input,[[2]+[3],[3],[2]+[3]],[50,100,150],[4,4,2],['mean','mean','mean'])
    couche,kernelBiasList,saver = discriminatorDense(generator,[150,300,150],'Lien')
    nbCouchesDsicriminateur = 15
    disc_faux,disc_vars,disc_saver = discriminator(generator,np.linspace(int(generator.get_shape()[1]),199,nbCouchesDsicriminateur+1,dtype=np.int)[1:],np.linspace(int(generator.get_shape()[-1]),3,nbCouchesDsicriminateur,dtype=np.int),'Faux')
    
    imageEntree = tf.summary.image("Image_entree",tf.cast(gen_input,dtype=tf.uint8))
    imageSortie = tf.summary.image("Image_sortie",tf.cast(disc_faux,dtype=tf.uint8))
    
    print('Taille de sortie disc_faux : ',end='')
    print(disc_faux.get_shape())
    print('Taille de sortie disc_input : ',end='')
    print(disc_input.get_shape())
    
    difference = disc_faux-gen_input
    disc_loss = tf.reduce_mean(difference)
    
    maximumGradient = tf.reduce_max(difference)
    minimumGradient = tf.reduce_min(difference)
    meanGradient = tf.reduce_mean(difference)
    maximumSortie = tf.reduce_max(disc_faux)
    minimumSortie = tf.reduce_min(disc_faux)
    meanSortie = tf.reduce_mean(disc_faux)
  
    loss_saver_disc = tf.summary.scalar("Loss",disc_loss)
    maximumGradientSaver = tf.summary.scalar("Maximum_difference",maximumGradient)
    minimumGradientSaver = tf.summary.scalar("Minimum_difference",minimumGradient)
    meanGradientSaver = tf.summary.scalar("Moyenne_difference",meanGradient)
    maximumSortieSaver = tf.summary.scalar("Maximum_sortie",maximumSortie)
    minimumSortieSaver = tf.summary.scalar("Minimum_sortie",minimumSortie)
    meanSortieSaver = tf.summary.scalar("Moyenne_sortie",meanSortie)
    
    optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)
    
    saver =  disc_saver
    print("Trainable variables : "+str(tf.trainable_variables()))
    train_disc = optimizer_disc.minimize(disc_loss)
    init = tf.global_variables_initializer()
    
    tf_saver = tf.train.Saver()
    summary_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

    import os
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        if os.path.isdir('model/') == True and forceInit == False:
          print(os.listdir('./model'))
          tf_saver.restore(sess, 'model/model'+nom+'.ckpt')
        else :
          print('No previous training found...')
          init.run()
        print('Entrainement....')
        for i in range(0,51):
          gen_inpt, disc_inpt = next_batch(batch_size, noises,images,height)#Problèmes si gen_input
          _,dl = sess.run([train_disc,disc_loss], feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: True})
          summary_str = loss_saver_disc.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          summary_writer.add_summary(summary_str,i)
        
          maximumGradientValue = maximumGradient.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          minimumGradientValue = minimumGradient.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          meanGradientValue = meanGradient.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          maximumSortieValue = maximumSortie.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          minimumSortieValue = minimumSortie.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          meanSortieValue = meanSortie.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          summary_str = maximumGradientSaver.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          summary_writer.add_summary(summary_str,i)
          summary_str = minimumGradientSaver.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          summary_writer.add_summary(summary_str,i)
          summary_str = meanGradientSaver.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          summary_writer.add_summary(summary_str,i)
          summary_str = maximumSortieSaver.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          summary_writer.add_summary(summary_str,i)
          summary_str = minimumSortieSaver.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          summary_writer.add_summary(summary_str,i)
          summary_str = meanSortieSaver.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          summary_writer.add_summary(summary_str,i)
          
          print('Step %i: Discriminator Loss: %f' % (i, dl))
          print('Différence : max %f, min %f, mean %f' % (maximumGradientValue, minimumGradientValue,meanGradientValue))
          print('Sortie : max %f, min %f, mean %f' % (maximumSortieValue, minimumSortieValue,meanSortieValue))
          if i == 50:
            summary_str = imageEntree.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
            summary_writer.add_summary(summary_str,i)
            summary_str = imageSortie.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
            summary_writer.add_summary(summary_str,i)
        tf_saver.save(sess, 'model/model'+nom+'.ckpt')
summary_writer.close()

Taille de gen_input : (7, 199, 199, 3)
Taille de convGenerator0_split_0 : (7, 199, 199, 50)
Taille de convGenerator0_split_1 : (7, 199, 199, 50)
Taille de convGenerator1_split_0 : (7, 100, 100, 100)
Taille de convGenerator2_split_0 : (7, 100, 100, 150)
Taille de convGenerator2_split_1 : (7, 100, 100, 150)
Taille de denseLien_1 : (7, 50, 50, 150)
Taille de denseLien_2 : (7, 50, 50, 300)
Taille de denseLien_3 : (7, 50, 50, 150)
Taille de l'entrée du discriminateur : (7, 50, 50, 150)
Taille du noyau de deconvolution de deconvDiscriminatorFaux0 : 10
Taille de deconvDiscriminatorFaux0 : (7, 59, 59, 150)
Taille du noyau de deconvolution de deconvDiscriminatorFaux1 : 11
Taille de deconvDiscriminatorFaux1 : (7, 69, 69, 139)
Taille du noyau de deconvolution de deconvDiscriminatorFaux2 : 11
Taille de deconvDiscriminatorFaux2 : (7, 79, 79, 129)
Taille du noyau de deconvolution de deconvDiscriminatorFaux3 : 11
Taille de deconvDiscriminatorFaux3 : (7, 89, 89, 118)
Taille du noyau de deconvolution d

In [0]:

# You can change the directory name
LOG_DIR = 'tf_logs'

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

import os
if not os.path.exists(LOG_DIR):
  os.makedirs(LOG_DIR)
  
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6064 &'
    .format(LOG_DIR))

get_ipython().system_raw('./ngrok http 6006 &')

!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
height = 199
width = 199
channels = 3
forceInit = False

batch_size = 7
learning_rate_power = 2
learning_rate = 10**-learning_rate_power
i = 1
nom = 'Autodencodeur_N_'+'mean'+'_'+str(i)+'_10e-'+str(learning_rate_power)
from tensorflow.python import debug as tf_debug

tf.reset_default_graph()
import traceback

root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)
with tf.Graph().as_default():
  
    def denseLayer(input,nbNeurones,nom,start=False,end=False):
      couche = tf.layers.dense(input,nbNeurones,name=nom)
#       print('Taille de '+nom+' : '+str(couche.get_shape()))
      kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
      bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
      kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
      bias_saver = tf.summary.histogram("Bias/"+nom,bias)
      return couche,kernel,bias,kernel_saver,bias_saver
    
    def lrelu(x, leak=0.2): 
      return tf.maximum(x, leak * x)
    
    def convLayer(inpt,kernelListOrNot,num_filters,poolingSize,poolingType,nom,start=False,end=False,generator=False):
      def conv(inpt,kernel,num_filters,poolingSize,poolingType,nom,start=False,end=False,pooling=True):
        couche = tf.layers.conv2d(inpt, filters=num_filters, kernel_size=kernel,
             strides=1, padding='SAME',
             activation=tf.nn.relu, name=nom)
#         print('Taille de '+nom+' : '+str(couche.get_shape()))
        kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
        bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
        kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
        bias_saver = tf.summary.histogram("Bias/"+nom,bias)
        couche = tf.contrib.layers.batch_norm(couche, is_training=training, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope=nom+'_normalization')
        if pooling == True:
          if poolingType == 'mean':
            couche = tf.layers.average_pooling2d(lrelu(couche), pool_size=(poolingSize,poolingSize), strides=(2,2), padding='same')
          else:
            couche = tf.layers.max_pooling2d(lrelu(couche), pool_size=(poolingSize,poolingSize), strides=(2,2), padding='same')
        return couche,kernel,bias,kernel_saver,bias_saver
      if type(kernelListOrNot) == list:
        couche,kernel,bias,kernel_saver,bias_saver = conv(inpt,kernelListOrNot[0],num_filters,poolingSize,poolingType,nom+'_split_0',start,end,pooling=False)
        kernelList,biasList,kernel_saverList,bias_saverList = [kernel],[bias],[kernel_saver],[bias_saver]
        for i in range(1,len(kernelListOrNot)):
          couche,kernel,bias,kernel_saver,bias_saver = conv(couche,kernelListOrNot[i],num_filters,poolingSize,poolingType,nom+'_split_'+str(i),False,False if i != len(kernelListOrNot)-1 else end,pooling=False if i != len(kernelListOrNot)-1 else True)
          kernelList.append(kernel)
          biasList.append(bias)
          kernel_saverList.append(kernel_saver)
          bias_saverList.append(bias_saver)
        return couche,kernelList,biasList,kernel_saverList,bias_saverList
      else: 
        return conv(inpt,kernelListOrNot,num_filters,poolingSize,poolingType,nom,start,end)
      
    def deconvLayer(inpt,sizeListOrNot,num_filters,nom,start=False,end=False):
      """Ceci est une documentation"""
      def deconv(inpt,size,num_filters,nom,start=False,end=False):
        def calculateParameters(precDimension,outputSize, strides): # Ref : https://adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/
#           print(precDimension)
          return outputSize-(precDimension-1)*strides
        kernelSize = calculateParameters(inpt.get_shape()[1],size,1)
#         print('Taille du noyau de deconvolution de '+nom+' : ' +str(kernelSize))
        couche = tf.layers.conv2d_transpose(inpt, filters=num_filters, kernel_size=(kernelSize,kernelSize),
             strides=1, padding='VALID',
             activation=tf.nn.relu, name=nom)
#         print('Taille de '+nom+' : '+str(couche.get_shape()))
        kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
        bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
        kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
        bias_saver = tf.summary.histogram("Bias/"+nom,bias)
        couche = tf.contrib.layers.batch_norm(couche, is_training=training, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope=nom+'_normalization')
        
        return couche,kernel,bias,kernel_saver,bias_saver
      if type(sizeListOrNot) == list:
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconv(inpt,sizeListOrNot[0],num_filters,poolingSize,poolingType,nom+'_split_0',True,False)
        kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList = [kernel],[bias],[kernel_saver],[bias_saver],[sortieImage]
        for i in range(1,len(sizeListOrNot)):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconv(couche,sizeListOrNot[i],num_filters,nom+'_split_'+str(i),False,False if i != len(sizeListOrNot)-1 else True)
          kernelList.append(kernel)
          biasList.append(bias)
          kernel_saverList.append(kernel_saver)
          bias_saverList.append(bias_saver)
        return couche,kernelList,biasList,kernel_saverList,bias_saverList
      else: 
        return deconv(inpt,sizeListOrNot,num_filters,nom,start,end)
    def generateur(inpt,kernels,num_filters,pooling,poolingTypes):
      """
      Description générale : Fonction créant le générateur, encodeur d'informations de l'image
      Entree : 
        inpt, couche d'entrée, ici ce sera l'image bruitée
        kernels, les différents noyaux de convolution, se présente soit sous forme de 
                liste simple soit sous forme d'une double liste. 
                Chaque sous liste représente un noyau de convolution 
                trop gros pour être réalisé en une couche et qui est donc séparé en sous-couches
        num_filters, la taille de filtre pour la couche (en gardant le même nombre de filtre
                pour chaque sous-couche si on a séparé la couche en sous-couches)
        poolingSize, les tailles des noyaux des couche de pooling
        poolingType, le type de pooling utilisé : SAME en générale
      Sortie : 
        L'image encodée (même si ce n'est plus réellement une image maintenant)
      """
      kernelBiasList,saver= [],[]
      def recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,kernelBiasList,saver):
        if type(kernel) == list:
          for k in kernel:
            kernelBiasList.append(k)
          for b in bias:
            kernelBiasList.append(b)
          for ks in kernel_saver:
            saver.append(ks)
          for bs in bias_saver:
            saver.append(bs)
        else:
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
        return None #Utilise la mutabilité des listes
      couche,kernel,bias,kernel_saver,bias_saver = convLayer(inpt,kernels[0],num_filters[0],pooling[0],poolingTypes[0],'convGenerator0',start=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,kernelBiasList,saver)
      for i in range(1,len(kernels)-1):
        couche,kernel,bias,kernel_saver,bias_saver = convLayer(couche,kernels[i],num_filters[i],pooling[i],poolingTypes[i],'convGenerator'+str(i))
        recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,kernelBiasList,saver)

      couche,kernel,bias,kernel_saver,bias_saver = convLayer(couche,kernels[-1],num_filters[-1],pooling[-1],poolingTypes[-1],'convGenerator'+str(len(kernels)-1),end=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,kernelBiasList,saver)
      return couche,kernelBiasList,saver

    def discriminator(inpt,size,num_filters,ID):
      """
      Entrees : 
        inpt, couche d'entrée du discriminateur
        size, liste, ou liste de liste si décomposition les tailles de couche successives pour revenir à la taille initiale
        num_filters, le nombre de filtre par couches
        ID, id unique pour séparer deux potentiels discriminateurs
      Sortie :
        Couche traitée par le discriminateur
      """
      kernelBiasList,saver= [],[]
      print("Taille de l'entrée du discriminateur : ",end='')
      print(inpt.get_shape())
      def recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,kernelBiasList,saver):
        if type(kernel) == list:
          for k in kernel:
            kernelBiasList.append(k)
          for b in bias:
            kernelBiasList.append(b)
          for ks in kernel_saver:
            saver.append(ks)
          for bs in bias_saver:
            saver.append(bs)
        else:
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
        return None #Utilise la mutabilité des listes
      couche,kernel,bias,kernel_saver,bias_saver = deconvLayer(inpt,size[0],num_filters[0],'deconvDiscriminator'+ID+'0',start=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,kernelBiasList,saver)
      for i in range(1,len(size)-1):
        couche,kernel,bias,kernel_saver,bias_saver = deconvLayer(couche,size[i],num_filters[i],'deconvDiscriminator'+ID+str(i))
        recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,kernelBiasList,saver)

      couche,kernel,bias,kernel_saver,bias_saver = deconvLayer(couche,size[-1],num_filters[-1],'deconvDiscriminator'+ID+str(len(size)-1),end=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,kernelBiasList,saver)
      return couche,kernelBiasList,saver
    def discriminatorDense(inpt,neurones,ID):#ID car besoin nom unique pour chaque série de discriminator
        saver = []
        kernelBiasList = []
        sortiesImages = []
        couche = inpt
        for i,nb in enumerate(neurones):
          couche,kernel,bias,kernel_saver,bias_saver = denseLayer(couche,nb,'dense'+str(ID)+'_'+str(i+1))
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
        return couche,kernelBiasList,saver
      
    gen_input = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="input_noise")
    disc_input = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="input_clean")
    training = tf.placeholder_with_default(False, shape=(), name='training')
    print('Taille de gen_input : '+str(gen_input.get_shape()))
    
    import numpy as np
    
    generator,gen_vars,gen_saver = generateur(gen_input,[[2]+[3],[3],[2]+[3]],[50,100,150],[4,4,2],['mean','mean','mean'])
    couche,kernelBiasList,saver = discriminatorDense(generator,[150,300,150],'Lien')
    nbCouchesDsicriminateur = 15
    disc_faux,disc_vars,disc_saver = discriminator(generator,np.linspace(int(generator.get_shape()[1]),199,nbCouchesDsicriminateur+1,dtype=np.int)[1:],np.linspace(int(generator.get_shape()[-1]),3,nbCouchesDsicriminateur,dtype=np.int),'Faux')
    
    imageEntree = tf.summary.image("Image_entree",gen_input)
    imageSortie = tf.summary.image("Image_sortie",disc_faux)
    
    print('Taille de sortie disc_faux : ',end='')
    print(disc_faux.get_shape())
    print('Taille de sortie disc_input : ',end='')
    print(disc_input.get_shape())
    
    difference = disc_faux-gen_input
    disc_loss = tf.reduce_mean(difference)
    
    maximumGradient = tf.reduce_max(difference)
    minimumGradient = tf.reduce_min(difference)
    meanGradient = tf.reduce_mean(difference)
    maximumSortie = tf.reduce_max(disc_faux)
    minimumSortie = tf.reduce_min(disc_faux)
    meanSortie = tf.reduce_mean(disc_faux)
  
    loss_saver_disc = tf.summary.scalar("Loss",disc_loss)
    maximumGradientSaver = tf.summary.scalar("Maximum_difference",maximumGradient)
    minimumGradientSaver = tf.summary.scalar("Minimum_difference",minimumGradient)
    meanGradientSaver = tf.summary.scalar("Moyenne_difference",meanGradient)
    maximumSortieSaver = tf.summary.scalar("Maximum_sortie",maximumSortie)
    minimumSortieSaver = tf.summary.scalar("Minimum_sortie",minimumSortie)
    meanSortieSaver = tf.summary.scalar("Moyenne_sortie",meanSortie)
    
    optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)
    
    saver =  disc_saver
#     print("Trainable variables : "+str(tf.trainable_variables()))
    train_disc = optimizer_disc.minimize(disc_loss)
    init = tf.global_variables_initializer()
    
    tf_saver = tf.train.Saver()
    summary_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

    import os
    with tf.Session() as sess:
        sess = tf_debug.TensorBoardDebugWrapperSession(sess, 'ubuntu:6064')
        tf.global_variables_initializer().run()
        if os.path.isdir('model/') == True and forceInit == False:
          print(os.listdir('./model'))
          tf_saver.restore(sess, 'model/model'+nom+'.ckpt')
        else :
          print('No previous training found...')
          init.run()
        print('Entrainement....')
        
        gen_inpt, disc_inpt = next_batch(batch_size, noises,images,height)
        entree = imageEntree.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
        print()
        summary_writer.add_summary(entree,54)
        sortie = imageSortie.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
        summary_writer.add_summary(sortie,54)
summary_writer.close()

Taille de gen_input : (7, 199, 199, 3)
Taille de l'entrée du discriminateur : (7, 50, 50, 150)
Taille de sortie disc_faux : (7, 199, 199, 3)
Taille de sortie disc_input : (7, 199, 199, 3)
['modelEssai3.ckpt.data-00000-of-00001', 'modelEssai3.ckpt.index', 'modelEssai3.ckpt.meta', 'modelEssai4.ckpt.data-00000-of-00001', 'modelEssai4.ckpt.index', 'modelEssai4.ckpt.meta', 'modelEssai5.ckpt.data-00000-of-00001', 'modelEssai5.ckpt.index', 'modelEssai5.ckpt.meta', 'modelEssai7T.ckpt.data-00000-of-00001', 'modelEssai7T.ckpt.index', 'modelEssai7T.ckpt.meta', 'modelEssai8.ckpt.data-00000-of-00001', 'modelEssai8.ckpt.index', 'modelEssai8.ckpt.meta', 'modelEssai9.ckpt.data-00000-of-00001', 'modelEssai9.ckpt.index', 'modelEssai9.ckpt.meta', 'modelEssai10.ckpt.data-00000-of-00001', 'modelEssai10.ckpt.index', 'modelEssai10.ckpt.meta', 'modelEssai11_test.ckpt.data-00000-of-00001', 'modelEssai11_test.ckpt.index', 'modelEssai11_test.ckpt.meta', 'modelEssai11.ckpt.data-00000-of-00001', 'modelEssai11.ckpt

_Rendezvous: ignored